[View in Colaboratory](https://colab.research.google.com/github/ZhaoMatt/GAN-Art-Forger/blob/master/MNIST_CNN_Keras.ipynb)

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import pandas as pd
import numpy as npp

In [0]:
#read data
df_XY_train = pd.read_csv('sample_data/mnist_train_small.csv')
df_X_test = pd.read_csv('sample_data/mnist_test.csv')

In [0]:
df_XY_train[:10]

In [0]:
Y_train = df_XY_train['6'].values
X_train = df_XY_train.drop('6',axis=1).values

X_test = df_X_test.values

In [0]:
# Reshape image , Standardize , One-hot labels
img_rows, img_cols = 28,28
input_shape = (img_rows, img_cols,1)
num_classes = 10

In [0]:
X_train=X_train.reshape(X_train.shape[0], img_rows, img_cols,1).astype('float32')/255
Y_train = keras.utils.to_categorical(Y_train,num_classes)


In [0]:
# Train model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [0]:
print(model.summary())

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(), 
              metrics=['accuracy'])

In [0]:
from keras.callbacks import TensorBoard
tb = TensorBoard('log_dir')

In [0]:
!pip install tensorboardcolab

In [0]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback

tbc=TensorBoardColab()

In [0]:
history = model.fit(X_train,Y_train,batch_size=128,epochs=25,verbose=1,callbacks=[TensorBoardColabCallback(tbc)])

In [0]:
history.history['acc'][10]  

In [0]:
TensorBoard('log_dir')

In [0]:
#data from Kaggle
X_test = pd.read_csv('test.csv').values
X_test = X_test.reshape(X_test.shape[0],img_rows, img_cols,1).astype('float32')/255
Y_predict = model.predict_classes(X_test)


In [0]:
X_test.shape

In [0]:
import numpy as np
predict = np.column_stack((np.arange(1,28001),Y_predict))

In [0]:
np.savetxt("predict.csv",predict,fmt='%i',delimiter=',',header='ImageId,Label',comments='')

In [0]:
!nvidia-smi